### Below cell demonstrates how to we get connect Oracle DB. 

* First, we import libraries. 
* Second, there is a fucntion regarding DB configiration.
* Usually in work there are two DB which names are Dev and Prod. We work in dev then we take our development to the prod. So this func have 2 options about that we can easly use. 

In [1]:
import pandas as pd
import cx_Oracle as ora
import sqlalchemy as sa

def get_database_engine(database_name):
    database_config={
        "prod": {
            "user": "user_name",
            "password": "db_password",
            "dsn": ora.makedsn('dns_adress', "port", service_name='db_name')
        },
        "dev": {
            "user": "user_name",
            "password": "db_password",
            "dsn": ora.makedsn('dns_adress', "port", service_name='db_name')
        },
    }
    return sa.create_engine(f'oracle+cx_oracle://{database_config[database_name]["user"]}:{database_config[database_name]["password"]}@{database_config[database_name]["dsn"]}')




In [ ]:
# After we create db_engine we can use it for writing DF to db or getting data from db to DF.

# There are two param about schema and which db we gonna work. 
schema = "which schema for writing the table"
which_db = "prodaction or development" 
if_exists = "it can be replace or append. If replace then it will clear all old data and replace it to new. If append then it will append new data to already exist one."

# The job of this func is write avaliable DataFrame to DB.
def write_to_database(df, table_name):
    with get_database_engine(which_db).connect() as connection:
        _dtype={c:sa.types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes=='object'].tolist()}
        _dtype.update({c:sa.types.NUMERIC(16,2) for c in df.columns[(df.dtypes=='float64') | (df.dtypes=='int64')].tolist()})
        df.to_sql(table_name, connection, schema=schema, index=False, if_exists=if_exists, dtype=_dtype)
        connection.close()
        connection.engine.dispose()
        
# And second func gets data from db to our DataFrame Object. 
with get_database_engine(which_db).connect() as connection:
    df=pd.read_sql("""select /*+parallel(8)*/ * from customers""", connection)
    connection.close()
    connection.engine.dispose()

In [ ]:
# As I said, we usually work two different db (dev, prod) and we might want to transfer data one to another. 
## Our func can do that too but it can be a problem.there might be a problem about that. Sometimes a column can full empty and it make a problem which is about to data type. 
## If we are trying to data transfer one db to another db in oracle, there is always all_tab_columns and we can use it for getting right column types. 


table_name_from_db = "''"

ower = "'DMFACT'"
# owner represents table scheme. 

with get_database_engine("prod").connect() as connection:
    df2=pd.read_sql("""SELECT COLUMN_NAME,DATA_TYPE,DATA_LENGTH ,DATA_PRECISION FROM all_tab_columns WHERE TABLE_name  ="""+str(table_name_from_db)+"""AND owner ="""+str(ower)+"""ORDER BY COLUMN_ID   """, connection)
    _dtype={c:sa.types.VARCHAR(df2[df2["column_name"]==str(c).upper()]["data_length"].values[0]) for c in df.columns[df.dtypes=='object'].tolist()}
    _dtype.update({c:sa.types.NUMERIC(16,2) for c in df.columns[(df.dtypes=='float64') | (df.dtypes=='int64')].tolist()})
    df.to_sql(".....", connection, schema=schema, index=False, if_exists=if_exists, dtype=_dtype)
    connection.close()